In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use("ggplot")
from cycler import cycler

- https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html
- https://matplotlib.org/stable/tutorials/colors/colors.html
- https://matplotlib.org/stable/tutorials/text/annotations.html#plotting-guide-annotation

## Chainage plot tool

## User settings
The following parameters need to be set by the user

In [ ]:
# Mandatory
PLOT_OUTPUTS_FC1_LIST = [
    r"BaselineGIS1Ammonia",
    r"BaselineGIS1BOD",
    r"BaselineGIS1Nitrate",
    r"BaselineGIS1Phosphate",
    r"BaselineGIS1Total_Phosphorus",
    r"BaselineGIS1DO_9924",
]
PLOT_OUTPUTS_FC2_LIST = [
    r"optionBGIS1Ammonia",
    r"optionBGIS1BOD",
    r"optionBGIS1Nitrate",
    r"optionBGIS1Phosphate",
    r"optionBGIS1Total_Phosphorus",
    r"optionBGIS1DO_9924",
]
RIVER_FC_LIST = [r"Thames (Evenlode to Thame)",]

# Styling
TARGET_FLAG = False  # True/False
TARGET_ANNOTATE = True  # True/False
ANNOTATE_FILTER = 0.001  # mg/l diff (Set to 0 for no filter)
FIG_SIZE = 10
ARROW_LENGTH = 0
LEGEND_LOC = "upper left"  # Supported legend values are: 'best', 'upper right', 'upper left',
# 'lower left', 'lower right', 'right', 'center left', 'center right',
# 'lower center', 'upper center', 'center'

### Code

In [ ]:
# Loop through rivers and determinands
for river in RIVER_FC_LIST:
    for det1, det2 in zip(PLOT_OUTPUTS_FC1_LIST, PLOT_OUTPUTS_FC2_LIST):

        # Carry out spatial selection on plot outputs using the river (2m intersection distance)
        arcpy.management.SelectLayerByLocation(
            det1, "INTERSECT", river, "2 Meters", "NEW_SELECTION", "NOT_INVERT"
        )
        arcpy.management.SelectLayerByLocation(
            det2, "INTERSECT", river, "2 Meters", "NEW_SELECTION", "NOT_INVERT"
        )

        # Read feature class as dataframe
        df1 = pd.DataFrame.spatial.from_featureclass(det1)
        df2 = pd.DataFrame.spatial.from_featureclass(det2)

        columns = [
            "OBJECTID",
            "ReachNo",
            "MeanConc",
            "LCLimMnCon",
            "UCLimMnCon",
            "CalcValQ90",
            "LCLimPer90",
            "UCLimPer90",
            "CalcValQ95",
            "LCLimPer95",
            "UCLimPer95",
            "CalcValQ99",
            "LCLimPer99",
            "UCLimPer99",
            "FeatName",
            "US_DS_Feat",
            "ReachName",
            "ObsConc",
            "ObsConcUCL",
            "ObsConcLCL",
            "ObsQ90Conc",
            "ObsQ90ConcUCL",
            "ObsQ90ConcLCL",
            "ObsQ95Conc",
            "ObsQ99Conc",
            "TargetMean",
            "Target90",
            "DetNo",
            "SWConc",
            "IMConc",
            "INConc",
            "MIConc",
            "LSConc",
            "ARConc",
            "HWConc",
            "URConc",
            "ATConc",
            "BGConc",
            "STConc",
            "LKConc",
            "DConc",
            "TargetHigh",
            "TargetGood",
            "TargetMod",
            "TargetPoor",
            "EA_WB_ID",
            "DiffConc",
            "PntConc",
            "CalStatus",
            "CalScore",
            "DISHeadKM",
            "DISPOINTKM",
        ]

        # Filter necessary columns
        df1 = df1[columns].copy()
        df2 = df2[columns].copy()

        # Sort per reach ascending
        df1.sort_values(["ReachNo", "OBJECTID"], inplace=True)
        df2.sort_values(["ReachNo", "OBJECTID"], inplace=True)

        # Calculate cummulative distance
        df1["DISTANCE"] = df1["DISPOINTKM"].cumsum()
        df2["DISTANCE"] = df2["DISPOINTKM"].cumsum()

        # Calculate diff in concentration
        df1["DIFF_CONC"] = df1["MeanConc"].diff().fillna(0)
        df2["DIFF_CONC"] = df2["MeanConc"].diff().fillna(0)

        # Remove first point next to headwater
        # TODO This needs to happen only if the first point is a headwater
#         df = df[1:].copy()

        # Set distance as index for x axis
        df1.set_index("DISTANCE", inplace=True)
        df2.set_index("DISTANCE", inplace=True)

        # Get rid of 0s in Observed concentrations
        df1["ObsConc"].replace(0, np.NaN, inplace=True)
        df1["ObsConcLCL"].replace(0, np.NaN, inplace=True)
        df1["ObsConcUCL"].replace(0, np.NaN, inplace=True)
        df2["ObsConc"].replace(0, np.NaN, inplace=True)
        df2["ObsConcLCL"].replace(0, np.NaN, inplace=True)
        df2["ObsConcUCL"].replace(0, np.NaN, inplace=True)

        # Rename sectors for a better legend
        apportionment_cols = {
            "SWConc": "Sewage",
            "IMConc": "Intermittent",
            "INConc": "Industry",
            "MIConc": "Mines",
            "LSConc": "Livestock",
            "ARConc": "Arable",
            "HWConc": "Highways",
            "URConc": "Urban",
            "ATConc": "Atmospheric",
            "BGConc": "Background",
            "STConc": "Septic tanks",
            "LKConc": "Lakes",
        }
        
        # Prepare colorscale
        apportionment_colormap = cycler(
            "color",
            [
                "0.1",
                "0.5",
                "r",
                "b",
                "yellowgreen",
                "yellow",
                "g",
                "purple",
                "olive",
                "pink",
                "gold",
                "orange",
                "lightsteelblue"
            ],
        )
        
        # Get metadata
        DETERMINAND = det1.split("GIS1")[1]
        SCENARIO = det1.split("GIS1")[0]
        SCENARIO2 = det2.split("GIS1")[0]
        
        # Add reach diffuse if needed
        # TODO Need to rewrite the dict without any other diffuse sectors!
        if DETERMINAND in ["Ammonia", "BOD", "DO_9924"]:
            apportionment_cols = {
                "SWConc": "Sewage",
                "IMConc": "Intermittent",
                "INConc": "Industry",
                "MIConc": "Mines",
                "DiffConc": "Reach Diffuse",
            }
            apportionment_colormap = cycler(
                "color",
                [
                    "0.1",
                    "0.5",
                    "r",
                    "b",
                    "lightsteelblue"
                ],
            )
        
        # Change column names
        df1.rename(apportionment_cols, inplace=True, axis=1)
        df2.rename(apportionment_cols, inplace=True, axis=1)

        # Set figure size
        plt.rcParams["figure.figsize"] = (FIG_SIZE, FIG_SIZE / (1.618 * 2.5 / 3))

        # Set colormap
        plt.rcParams["axes.prop_cycle"] = apportionment_colormap

        # Set up initial layout
        fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, sharey=True)

        ## Calibration plot
        # Simulated data
        df1.MeanConc.plot(ax=axes[0], c="k")
        df1.UCLimMnCon.plot(ax=axes[0], c="k", ls="--", alpha=0.75)
        df1.LCLimMnCon.plot(ax=axes[0], c="k", ls="--", alpha=0.75)

        # Observed data
        df1.ObsConc.plot(ax=axes[0], c="b", marker="o")
        df1.ObsConcUCL.plot(ax=axes[0], c="b", marker=".")
        df1.ObsConcLCL.plot(ax=axes[0], c="b", marker=".")

        ## Apportionment plot 1
        # Data
        df1[apportionment_cols.values()].replace(0, np.NaN).plot.area(ax=axes[1], lw=0)

        # Targets
        if TARGET_FLAG:
            df1.TargetPoor.plot(ax=axes[1], c="r", ls="--", alpha=0.5)
            df1.TargetMod.plot(ax=axes[1], c="orange", ls="--", alpha=0.5)
            df1.TargetGood.plot(ax=axes[1], c="g", ls="--", alpha=0.5)
            df1.TargetHigh.plot(ax=axes[1], c="b", ls="--", alpha=0.5)

        ## Apportionment plot 2
        # Data
        df2[apportionment_cols.values()].replace(0, np.NaN).plot.area(ax=axes[2], lw=0)

        # Targets
        if TARGET_FLAG:
            df2.TargetPoor.plot(ax=axes[1], c="r", ls="--", alpha=0.5)
            df2.TargetMod.plot(ax=axes[1], c="orange", ls="--", alpha=0.5)
            df2.TargetGood.plot(ax=axes[1], c="g", ls="--", alpha=0.5)
            df2.TargetHigh.plot(ax=axes[1], c="b", ls="--", alpha=0.5)        
        
        # Titles
        fig.suptitle(
            # TODO Redo title
            f"{DETERMINAND} calibration (top) and source apportionment for " +
            f"scenarios {SCENARIO} (middle) and {SCENARIO2} (bottom) for " +
#             f"reaches {df1.ReachNo.min()}-{df1.ReachNo.max()}",
            f"\n the Thames river (Evenlode to Thame)",
            size=12,
            ha="center",
            y=0.98,
        )
        # axes[0].set_title("Calibration")
        # axes[1].set_title("Apportionment")

        # Make sure axis starts at 0
        axes[0].set_ylim(bottom=0)
        axes[1].set_ylim(bottom=0)

        # Axis labels
        axes[0].set_ylabel("Concentration (mg/L)")
        axes[1].set_ylabel("Concentration (mg/L)")
        axes[2].set_ylabel("Concentration (mg/L)")
        axes[2].set_xlabel("Distance (km)")

        # Activate legend
        axes[0].legend(
            labels=[
                "Sim. Mean",
                "Sim. Mean UCL",
                "Sim. Mean LCL",
                "Obs. Mean",
                "Obs. Mean UCL",
                "Obs. Mean LCL",
            ]
        )
        axes[1].legend(ncol=2, loc=LEGEND_LOC)  # change to 'upper right' if needed
        axes[2].legend(ncol=2, loc=LEGEND_LOC)  # change to 'upper right' if needed

        # Annotations
        if TARGET_ANNOTATE:
            # Apply filter
            mask = (df1["DIFF_CONC"] > ANNOTATE_FILTER) & (df1.US_DS_Feat == "d-s")
            mask2 = df1["FeatName"].isin(["SESRO Intake Outake Opt B", "Additional Reach"])
            mask = mask | mask2
            # Draw labels
            count = 0
            max_conc = axes[1].get_ylim()[1]
            for dis, row in df1[mask].iterrows():
                axes[1].annotate(
                    row.FeatName,
                    xy=(dis, row.MeanConc),
                    xycoords="data",
                    xytext=(dis, 
                            max_conc - 0.15 * max_conc * (count)),
                    textcoords="data",
                    va="top",
                    ha="center",
                    arrowprops=dict(arrowstyle="->", connectionstyle="arc3", color="k"),
                    bbox=dict(boxstyle="square, pad=0.3", fc="1", ec="0.5", lw=1, alpha=0.5),
                )
                count += 1
            # Apply filter
            mask = (df2["DIFF_CONC"] > ANNOTATE_FILTER) & (df2.US_DS_Feat == "d-s")
            mask2 = df2["FeatName"].isin(["SESRO Intake Outake Opt B", "Additional Reach"])
            mask = mask | mask2
            # Draw labels
            count = 0
            max_conc = axes[2].get_ylim()[1]
            for dis, row in df2[mask].iterrows():
                axes[2].annotate(
                    row.FeatName,
                    xy=(dis, row.MeanConc),
                    xycoords="data",
                    xytext=(dis, 
                            max_conc - 0.15 * max_conc * (count)),
                    textcoords="data",
                    va="top",
                    ha="center",
                    arrowprops=dict(arrowstyle="->", connectionstyle="arc3", color="k"),
                    bbox=dict(boxstyle="square, pad=0.3", fc="1", ec="0.5", lw=1, alpha=0.5),
                )
                count += 1

        # Make everything tidy
        plt.tight_layout()

        # Create Figures folder in project folder if it doesn't exist
        p = arcpy.mp.ArcGISProject("CURRENT")
        fig_folder = os.path.join(p.homeFolder, "Figures")
        if not os.path.exists(fig_folder):
            os.mkdir(fig_folder)

        # Save figure and show
        fig_name = f"{det2}_outputs_{df1.ReachNo.min()}_{df1.ReachNo.max()}.png"
        fig_path = os.path.join(fig_folder, fig_name)
        print(f"Created {fig_name}")
        plt.savefig(fig_path)
        plt.close()

Created optionBGIS1Ammonia_outputs_285_286.png
Created optionBGIS1BOD_outputs_285_286.png
Created optionBGIS1Nitrate_outputs_285_286.png
Created optionBGIS1Phosphate_outputs_285_286.png
Created optionBGIS1Total_Phosphorus_outputs_285_286.png
Created optionBGIS1DO_9924_outputs_285_286.png


**TODO**
- Transform into a function for batch run
- Could be interesting to plot cal score
- Could also annotate EA_WB_ID and confluences
- Add functionality to include annotated vertical labels (for tributaries)